Opis:

słuchamy aż usłyszymy że ktoś do nas nadaje (rozróżniam 3 stany - ciszę, częstotliwość A i częstotliwość B)

wtedy wczytuję całą ramkę i odczytuję wiadomość w niej zawartą

ramka ma też bit który mówi, czy to jest ostatnia ramka wiadomości, czy nie

jeśli to nie jest ostatnia ramka, to słuchamy dalej

In [3]:
import wave
import pulseaudio as pa
import numpy as np
import uuid
from Encoder import Encoder

class Listener(object):

    @staticmethod
    def find_frequency(framerate, samples):
        """
            znajduje częstotliwość (przepraszam za redundancję w kodzie)
        """
        with pa.simple.open(direction=pa.STREAM_RECORD, 
                            format=pa.SAMPLE_S16LE, 
                            rate=framerate, 
                            channels=1) as recorder:
            wave = np.fromstring(recorder.read(2 * samples), np.int16)
            w = np.fft.fft(wave)
            freqs = np.fft.fftfreq(len(w))
            res = abs(freqs[np.argmax(np.abs(w))] * framerate)
            return res
    
    @staticmethod
    def get_frequency(wave, framerate=44100):
        """
            bierze falkę i zwraca częstotliwość
        """
        w = np.fft.fft(wave)
        freqs = np.fft.fftfreq(len(w))
        return abs(freqs[np.argmax(np.abs(w))] * framerate)
    
    @staticmethod
    def frequency_to_bit(freq, freqs):
        """
            krótka heura na stwierdzenie czy 
            częstotliwość jest jedną z tych 
            na których nasłuchujemy
        """
        if freq < freqs[0] * 0.75:
            return -1
        if freq < sum(freqs) / 2:
            return 0
        return 1
    
    def __init__(self, framerate=44100, frequencies=[1764,4410], bit_length=100):
        """
            konstruktor
        """
        self.framerate = framerate
        self.frequencies = frequencies
        self.bit_length = bit_length
        self.mac = ':'.join(("%012X" % uuid.getnode())[i:i+2] for i in range(0, 12, 2))

    def get_waves(self, samples, nbits):
        """
            słucha samples i zwraca listę falek (każda falka reprezentuje jeden z nbits bitów)
        """
        with pa.simple.open(direction=pa.STREAM_RECORD, 
                            format=pa.SAMPLE_S16LE, 
                            rate=self.framerate, 
                            channels=1) as recorder:
            wave = np.fromstring(recorder.read(2 * samples * nbits), np.int16)
            waves = [wave[(samples * a):(samples * (a + 1))] for a in xrange(nbits)]
            return waves
    
    def solve(self, package_size=100):
        """
            robi to, czego oczekujemy, czyli rozwiązuje problemy
            przyjmuje liczbę bitów wiadomości (bez opakowania) w pojedynczej paczce
            i słucha
            i słucha
            i słucha
            i zwraca to co wysłyszał
        """
        #powiększamy package_size o rozmiar zbędnego prefiksu i sufiksu
        package_size = (package_size * 5) / 4
        package_size += 660
        
        results = []
        
        while True:
            res = []
            ret = []
            while True:
                start_bit = Listener.find_frequency(self.framerate, self.bit_length)
                if Listener.frequency_to_bit(start_bit, self.frequencies) == 1:
                    ret.append(self.frequency_to_bit(start_bit, self.frequencies))
                    last_frequency = start_bit
                    while Listener.frequency_to_bit(last_frequency, self.frequencies) >= 0:
                        ple = self.get_waves(self.bit_length, package_size)
                        res.append(ple)
                        last_frequency = Listener.get_frequency(ple[-1])
                    break

            for i in xrange(len(res)):
                for j in xrange(len(res[i])):
                    wave = res[i][j]
                    if j == 0:
                        wave = wave[:(len(wave)/2)]
                    r = Listener.get_frequency(wave)
                    a = Listener.frequency_to_bit(r, self.frequencies)
                    ret.append(a)

            ret = Encoder.restore_message(ret)
            ret, bit = Encoder.get_data_from_message(ret)
            results.append(ret)
            if bit == "1":
                return ''.join(results)

Importing ipynb file : Encoder.ipynb


In [7]:
l = Listener(bit_length=50)
s = l.solve(package_size=1999)
print s
print len(s)
print Encoder.string_from_bits(s)

0101011101101001011101000110000101101010011000110110100101100101001000010010000001001110011000010111101001111001011101110110000101101101001000000111001101101001011001010010000001001010011000010110001101100101011010110010000001001100011000010111001101111010011000110111101001101111011010110010000001010011011101000110000101100011011010000111010101110010011100110110101101111001001000000101010101110010011011110110010001111010011010010110110001100101011011010010000001110011011010010110010100100000011001000111011101110101011001000111101001101001011001010111001101110100011001010110011101101111001000000111001101111010011011110111001101110100011001010110011101101111001000000111001101110100011110010110001101111010011011100110100101100001001000000111010001111001011100110110100101100001011000110010000001100100011110100110100101100101011101110110100101100101011000110111001101100101011101000010000001110011011110100110010101110011011000110110010001111010011010010110010101110011011010010110000101110100

Problem: ten pulseaudio dostaje retarda pod koniec sygnału.

Rozwiazanie: wczytujemy tyle bitow ile sie da (tj. dlugosc ramki którą slemy)

Nieudane próby rozwiazania:

1) Wysyłanie wiadomosci bit po bicie - 

zupelnie nie dziala

2) Wysyłanie wiadomosci w paczkach czasowch - 

czas jest slaby, bo rozne rzeczy zuzywaja czas i mamy rozstrojenie

3) Wysyłanie wiadomosci w paczkach mniejszych niz maksymalna dlugosc -

trzeba by dzielic wiadomosc na kawalki i do kazdego doklejac stratny sufiks, a to w zasadzie sie nie rozni od wysylania calej ramki i sluchania calej ramki, a jest mniej roboty